In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import requests
import json
import pprint

import pandas as pd

# fix ssl certificate (needed for MacOS sometimes)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

/Users/davidshaw/Projects/minWage/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# get html, convert to bs4 object
url = 'https://www.capfriendly.com/signings/all/all/all/1-15/0-15000000/11012000-11012003'
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, 'html.parser')

# # find table that contains data
# table_rows = soup.find('table', class_='sortablex tblcf tbl index').find('tbody').find_all('tr')

# # scrape data into dict
# team_and_link = {}
# for row in table_rows:
#     team_name = row.find('a').get_text()
#     team_link = row.find('a').get('href')[:-4]
#     team_and_link[team_name] = team_link

In [7]:
rows = soup.find('tbody').find_all('tr')

In [15]:
all_contracts = []
for row in rows:
    contract = []
    for td in row.find_all('td'):
        contract.append(td.get_text())
    all_contracts.append(contract)

In [16]:
all_contracts

[['Marián Gáborík',
  '21',
  'RW',
  'MIN',
  'Oct. 31, 2003',
  'Stnd (RFA)',
  '',
  '1-way',
  '3',
  '$5,579,000',
  '$1,859,667'],
 ['Todd Bertuzzi',
  '28',
  'RW',
  'VAN',
  'Oct. 26, 2003',
  'Stnd (UFA)',
  '',
  '1-way',
  '4',
  '$17,471,160',
  '$6,933,000'],
 ['Marc-André Fleury',
  '18',
  'G',
  'PIT',
  'Oct. 6, 2003',
  'ELC',
  '',
  '2-way',
  '3',
  '$2,182,400',
  '$727,467'],
 ['Patrice Bergeron',
  '18',
  'C',
  'BOS',
  'Oct. 6, 2003',
  'ELC',
  '',
  '2-way',
  '3',
  '$1,546,000',
  '$515,333'],
 ['Chris Drury',
  '26',
  'C',
  'BUF',
  'Aug. 28, 2003',
  'Stnd (UFA)',
  '',
  '2-way',
  '4',
  '$9,530,000',
  '$2,382,500'],
 ['Brad Richards',
  '23',
  'C, LW',
  'TBL',
  'Aug. 27, 2003',
  'Stnd (RFA)',
  '',
  '1-way',
  '3',
  '$5,801,000',
  '$1,933,667'],
 ['Milan Michalek',
  '18',
  'LW, RW',
  'SJS',
  'Aug. 18, 2003',
  'ELC',
  '',
  '2-way',
  '3',
  '$1,884,800',
  '$628,267'],
 ['Kimmo Timonen',
  '28',
  'D',
  'NSH',
  'Aug. 15, 2003',
  '

In [17]:
pd.DataFrame(all_contracts)

,0,1,2,3,4,5,6,7,8,9,10
0,Marián Gáborík,21,RW,MIN,"Oct. 31, 2003",Stnd (RFA),,1-way,3,"$5,579,000","$1,859,667"
1,Todd Bertuzzi,28,RW,VAN,"Oct. 26, 2003",Stnd (UFA),,1-way,4,"$17,471,160","$6,933,000"
2,Marc-André Fleury,18,G,PIT,"Oct. 6, 2003",ELC,,2-way,3,"$2,182,400","$727,467"
3,Patrice Bergeron,18,C,BOS,"Oct. 6, 2003",ELC,,2-way,3,"$1,546,000","$515,333"
4,Chris Drury,26,C,BUF,"Aug. 28, 2003",Stnd (UFA),,2-way,4,"$9,530,000","$2,382,500"
5,Brad Richards,23,"C, LW",TBL,"Aug. 27, 2003",Stnd (RFA),,1-way,3,"$5,801,000","$1,933,667"
6,Milan Michalek,18,"LW, RW",SJS,"Aug. 18, 2003",ELC,,2-way,3,"$1,884,800","$628,267"
7,Kimmo Timonen,28,D,NSH,"Aug. 15, 2003",Stnd (RFA),,1-way,4,"$6,750,000","$1,687,500"
8,Christian Ehrhoff,21,LD,SJS,"Jul. 23, 2003",ELC,,2-way,3,"$1,258,000","$419,333"
9,Sergei Fedorov,33,C,ANA,"Jul. 19, 2003",Stnd (UFA),,1-way,5,"$26,240,000","$5,248,000"
